## Training Data Prep

```sql
copy(
    select
      -- type,
      -- subtype           ,
      ts,
      to_timestamp(cast(ts as bigint)) as ts_timedate,
      user              ,
      text              ,
      -- client_msg_id     ,
      -- blocks            ,
      -- team              ,
      -- user_team         ,
      -- source_team       ,
      -- user_profile      ,
      -- inviter           ,
      -- edited            ,
      reactions         ,
      thread_ts,
      to_timestamp(cast(thread_ts as bigint)) as thread_ts_timedate,
      -- reply_count       ,
      -- reply_users_count ,
      -- latest_reply      ,
      -- reply_users       ,
      replies           ,
      -- is_locked         ,
      -- subscribed        ,
      -- last_read         ,
      parent_user_id    ,
      regexp_extract(filename, 'kaskada-slack-export/(.+)/\d{4}-\d{2}-\d{2}.json', 1) as channel
    from (
        select * from read_json_auto('kaskada-slack-export/*/*.json',
            format='array',
            filename=true,
            union_by_name=true)
    )
    where subtype is null
) to 'messages.jsonl' (FORMAT JSON)
;
```

In [15]:
## prompt is last 5-ish messages, completion is next user to write.

import json

file1 = open('/Users/eric.pinzur/Documents/messages.jsonl', 'r')

current_channel = ""
recent_messages = []
min_messages = 2
max_messages = 5
count = 0
while True:
    line = file1.readline()

    if not line:
        break

    data = json.loads(line)
    channel = data["channel"]
    text = data["text"]
    user = data["user"]
    replies = data["replies"]
    reactions = data["reactions"]
    between_message_separator = " \n\n\n "
    reverse_messages = True

    if channel != current_channel:
        current_channel = channel
        recent_messages = []
        print(f'New channel: {current_channel}')

    if len(recent_messages) > min_messages:
        messages = recent_messages.copy()
        if reverse_messages:
            messages.reverse()
        prompt = between_message_separator.join(messages)
        training_example = { "prompt": prompt, "completion": " " + user}
        print(training_example)
        count += 1
        if count > 10:
            break

    recent_messages.append(text)
    if len(recent_messages) > max_messages:
        recent_messages.pop(0)

file1.close()

New channel: team-api
{'prompt': 'Did some quick digging into release options for go binaries, and this looks like a pretty nice tool: \n\n\n Let me know if you need anything else for it \n\n\n <@U016TM9NXEY> I’m making various changes to your wren MR 3568', 'completion': ' ULJD5H2A2'}
{'prompt': '<https://goreleaser.com/customization/blob/> \n\n\n Did some quick digging into release options for go binaries, and this looks like a pretty nice tool: \n\n\n Let me know if you need anything else for it \n\n\n <@U016TM9NXEY> I’m making various changes to your wren MR 3568', 'completion': ' ULJD5H2A2'}
{'prompt': 'Has a bunch of options for publishing - one is to push to S3. So we could create a public bucket and publish into it pretty easily \n\n\n <https://goreleaser.com/customization/blob/> \n\n\n Did some quick digging into release options for go binaries, and this looks like a pretty nice tool: \n\n\n Let me know if you need anything else for it \n\n\n <@U016TM9NXEY> I’m making various 